In [134]:
# Importar librerías para manipular los datos.
import pandas as pd
import numpy as np
import re

In [135]:
#Importar archivo con la tabla definitiva y verificar estructura.
datos = pd.read_csv('./app/base_plataformas.csv', engine='python', decimal='.')
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Titulo        22998 non-null  object
 1   Duracion      22998 non-null  int64 
 2   Anio_estreno  22998 non-null  int64 
 3   Tipo          22998 non-null  object
 4   Genero        22998 non-null  object
 5   Actores       22998 non-null  object
 6   Plataforma    22998 non-null  object
dtypes: int64(2), object(5)
memory usage: 1.2+ MB


DESARROLLO DE CONSULTAS

Máxima duración según tipo de film (película/serie), por plataforma y por año: El request debe ser: get_max_duration(año, plataforma, [min o season])

In [58]:
def get_max_Duracion(anio,plataforma,tipo):
    lista_plataforma = ['Amazon','Disney','Hulu','Netflix']
    lista_tipo = ['min','season']
    if (plataforma in lista_plataforma) and (tipo in lista_tipo) and (anio in range(datos.Anio_estreno.min(),datos.Anio_estreno.max()+1)):
        if tipo == 'min':
            tipo_MoS = 'Movie'
            gmd = datos[datos['Plataforma'] == plataforma][datos['Anio_estreno'] == anio][datos['Tipo'] == tipo_MoS].groupby(['Plataforma', 'Tipo', 'Anio_estreno'])['Duracion'].idxmax()
            duracion = datos['Duracion'].get(gmd[0])
            titulo = datos['Titulo'].get(gmd[0])
            return (f'El Título es {titulo} y su duración es de {duracion} {tipo}')
        elif tipo == 'season':
            tipo_MoS = 'TV Show'
            gmd = datos[datos['Plataforma'] == plataforma][datos['Anio_estreno'] == anio][datos['Tipo'] == tipo_MoS].groupby(['Plataforma', 'Tipo', 'Anio_estreno'])['Duracion'].idxmax()
            duracion = datos['Duracion'].get(gmd[0]) 
            titulo = datos['Titulo'].get(gmd[0])
            return (f'El Título es {titulo} y su duración es de {duracion} {tipo}')
    else:
        return (f'Los parámetros ingresados son erróneos.')
    
# La función analiza inicialmente si los parámetros son correctos, sino devuelve un mensaje de error. Luego extrae el ID con el valor máximo del dato correspondiente a cada parámetro
# en función a la columna Duración. Por último devuelve el dato ubicado en cada índice de los parámetros. 

In [60]:
get_max_Duracion(2018,'Hulu','min')

C:\Users\emman\AppData\Local\Temp\ipykernel_10776\3124994940.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  gmd = datos[datos['Plataforma'] == plataforma][datos['Anio_estreno'] == anio][datos['Tipo'] == tipo_MoS].groupby(['Plataforma', 'Tipo', 'Anio_estreno'])['Duracion'].idxmax()


'El Título es The House That Jack Built y su duración es de 151 min'

Cantidad de películas y series (separado) por plataforma El request debe ser: get_count_plataform(plataforma)

In [136]:
def get_count_plataform(plataforma):
    lista_plataforma = ['Amazon','Disney','Hulu','Netflix']
    if (plataforma in lista_plataforma):
        gcp = datos[datos['Plataforma'] == plataforma]
        gcp = gcp.groupby(['Plataforma'])['Tipo'].value_counts()
        return gcp
        
    else:
        return (f'Los parámetros ingresados son erróneos.')
    
# La función analiza inicialmente si el parámetro es correctos, sino devuelve un mensaje de error. Primeramente arma una tabla reducida, con los datos correspondientes
# a la plataforma ingresada en el parámetro. Por último agrupa según los valores dentro de la columna Tipo y cuenta las repeticiones de cada uno, devolviendo esa información.

In [137]:
get_count_plataform('Disney')

Plataforma  Tipo   
Disney      Movie      1052
            TV Show     398
Name: Tipo, dtype: int64

In [126]:
gcp = datos[datos['Plataforma'] == 'Hulu']
gcp = gcp.groupby(['Plataforma'])['Tipo'].value_counts()
#gcp = pd.DataFrame(gcp)
print (gcp)

Plataforma  Tipo   
Hulu        TV Show    1589
            Movie      1484
Name: Tipo, dtype: int64


Cantidad de veces que se repite un género y plataforma con mayor frecuencia del mismo. El request debe ser: get_listedin('genero')
Como ejemplo de género pueden usar 'comedy', el cuál deberia devolverles un cunt de 2099 para la plataforma de amazon.

In [89]:
def get_listedin(genero):
    gl = datos[datos.Genero.str.contains(genero, case=False)].groupby(by=['Plataforma']).Titulo.count()
    gl = pd.DataFrame(gl)
    gl.reset_index(inplace=True)
    gl.sort_values(by='Titulo', inplace=True, ascending=False)
    gl.reset_index(inplace=True, drop=True)
    return ('El género ') + str(genero) + (' se repite ') + str(gl.loc[1,'Titulo']) + (' veces en la plataforma ') + str(gl.loc[1,'Plataforma'])

# La función recorre la columna Genero y busca el parámetro ingresado. Agrupa los resultados por plataforma y realiza un conteo de las repeticiones. 
# El resultado del paso anterior se reordena de forma decreciente y se retorna el primer valor: la plataforma con mayor número de repeticiones. gl.iloc[0]

In [90]:
get_listedin ('comedy')

'El género comedy se repite 667 veces en la plataforma Hulu'

Actor que más se repite según plataforma y año. El request debe ser: get_actor(plataforma, año)

In [75]:
def get_actor(plataforma,anio):
    lista_plataforma = ['Amazon','Disney','Hulu','Netflix']
    lista_tipo = ['min','season']
    if (plataforma in lista_plataforma) and (anio in range(datos.Anio_estreno.min(),datos.Anio_estreno.max()+1)):
        actores_lista = []
        ga = datos.query(f"Plataforma == '{plataforma}' and Anio_estreno ==  {anio}").Actores.tolist()
        for i in range(len(ga)):
            actores_temp = ga[i].split(",")
            for j in range(len(actores_temp)):
                if actores_temp[j] != 'Sin dato':
                    actores_lista.append(actores_temp[j])
        actores_dicc = dict(zip(actores_lista,map(lambda x: actores_lista.count(x),actores_lista)))
        actor_max = max(actores_dicc, key=actores_dicc.get)
        actor_max_aparece = actores_dicc.get(actor_max)
        return f'Es {actor_max} con {actor_max_aparece} apariciones.'
    else:
        return (f'Los parámetros ingresados son erróneos.')
    
# La función analiza inicialmente si los parámetros son correctos, sino devuelve un mensaje de error.. Como primer paso arma una tabla reducida en función de la plataforma
# y año ingresados. Luego arma una lista con todos los nombres de actores que encuentra al recorrer la columna indicada. Finalmente hace un conteo de las repeticiones de cada
# nombre y devuelve aquel que más se repite. 

In [76]:
get_actor ("Netflix",2018)

'Es  Andrea Libman con 8 apariciones.'